In [12]:
# PATH_NAME = 'C:/Users/gkstk/OneDrive/Desktop/SangMin/Github/AI/data/'
# PATH_NAME2 = 'C:/Users/gkstk/OneDrive/Desktop/SangMin/Github/AI/'


In [ ]:
PATH_NAME = 'C:/Users/woobi/Documents/LearningFoodPreferences2/data/'
PATH_NAME2 = 'C:/Users/woobi/Documents/LearningFoodPreferences2/'

In [13]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\gkstk\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [14]:
import pandas as pd
import numpy as np
import re
from konlpy.tag import Okt
from gensim.models import KeyedVectors
from sklearn.metrics.pairwise import cosine_similarity
from collections import defaultdict
import pickle
import math
from collections import Counter

In [15]:
model = KeyedVectors.load(PATH_NAME + "한국어_음식모델_한상민.kv", mmap='r')

with open(PATH_NAME + 'wweia_synonym_cats.pickle', 'rb') as handle:
  wweia_synonym_cats = pickle.load(handle)
  
wweia_food_categories = pd.read_csv(PATH_NAME + 'wweia_food_categories_addtl.csv')
wweia_data = pd.read_csv(PATH_NAME + 'wweia_data.csv')

In [16]:
# Getting a sample of what an embedding looks like
model["무지개떡"]

array([-1.70645416e-02, -3.61461681e-03,  8.80778488e-03, -1.13084717e-02,
        1.48727577e-02, -4.35531605e-03, -4.19942942e-03,  1.63003020e-02,
       -9.29594878e-03,  2.57301726e-05, -1.20856445e-02, -1.98923294e-02,
       -6.91276789e-03,  1.94507930e-02,  7.37692462e-03, -1.49570396e-02,
        5.67844929e-03, -8.31672829e-03, -1.82552461e-03, -9.48544033e-03,
        1.92627385e-02, -6.15636306e-03, -3.39669385e-03,  7.84676615e-03,
        6.40838360e-03,  1.17731579e-02,  7.05219153e-03, -7.66839506e-03,
       -2.61669792e-03, -8.55937880e-03,  4.93752630e-03,  8.54510628e-03,
       -1.34234093e-02, -1.25254430e-02, -6.64528925e-03, -8.66752956e-03,
       -3.22204502e-03,  2.94499972e-04,  5.67830401e-04, -1.99864544e-02,
       -1.42542869e-02, -5.63145429e-03,  5.94050810e-03, -8.85985047e-03,
        1.78560801e-02, -7.92126358e-03, -1.00456756e-02, -3.88140877e-04,
        7.83004425e-03,  1.01383701e-02,  4.05120663e-03, -1.22793429e-02,
       -1.59502551e-02, -

In [17]:
# Load embeddings
wweia_embeddings = np.loadtxt(PATH_NAME + 'word_embeddings.csv', delimiter = ",")
wweia_embeddings.shape

(3710, 100)

In [18]:
stop_words = ['가', '걍', '것', '고', '과', '는', '도', '들', '등', '때', '로', '를', '뿐', '수', '아니', '않', '없', '에', '에게', '와', '으로', '은', '의', '이', '이다', '있', '자', '잘', '좀', '하다', '한', '조각', '개', '것', '대', '소' ,'단계', '등급', '포함', '미니', '개입']

In [19]:
# Only keep words in phrase that are in food_words

def reduce_with_food_words(rough_phrase):
  korean_string = re.sub("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", " ", rough_phrase)
  okt = Okt()
  token = okt.morphs(korean_string, stem=True)
  return token

In [20]:
# Format of food names seems to be (Optional) Company, Food, Details about food
# run function to guess if first or second comma phrase has Food

def process_food_log(curr_log, wweia_synonym_cats):
  curr_log['predicted_categories_number'] = 0
  curr_log['predicted_categories_words'] = ""
  curr_log['max_cosim_score'] = 0
  curr_log['most_sim_food'] = ""
  curr_log['reciprocal_rank'] = 0.0
  curr_log['sym_reciprocal_rank'] = 0.0

  for i in range(curr_log.shape[0]):
    food_name = curr_log.loc[i, 'Food Name']
    pre_embedding = reduce_with_food_words(food_name)

    word_embed = np.zeros(shape = (1, len(model["불고기"])))
    if len(pre_embedding) > 0:
      
      num_words = 0
      for word in pre_embedding:
        word = word.lower()
        
        if word in model:
          num_words += 1
          word_embed += model[word]

      if num_words != 0:
        word_embed /= num_words
    
    # print("음식 이름")
    # print(food_name)
    # print("프리임베딩")
    # print(pre_embedding)
    # print(word_embed)
    # print('\n')
    
    similarities = cosine_similarity(word_embed, wweia_embeddings)
    to_keep = np.sort(similarities, axis=1)
    to_keep_args = np.argsort(similarities, axis=1)
    indices = np.flip(to_keep_args, axis = 1)
    
    sym_rank = 1000000
    for index in range(indices.shape[1]):
      true_cat = curr_log.loc[i, 'wweia_food_category_code']
      if math.isnan(true_cat): continue
      if wweia_data.loc[indices[0,index], 'NO'] == true_cat:
        rank = index
        rr = 1 / (rank+1)
        if sym_rank > rank:
          sym_rank = rank
          sym_rr = 1 / (rank+1)
        break
      else:
        
        if wweia_data.loc[indices[0,index], 'NO'] in wweia_synonym_cats[true_cat]:
          if sym_rank > index:
            sym_rank = index
            sym_rr = 1 / (index+1)
    most_sim_food_index = indices[0,0]

    most_sim_food_row = wweia_data.iloc[most_sim_food_index, :]
    highest_cat_num = most_sim_food_row['NO']
    highest_cat_words = wweia_food_categories.loc[wweia_food_categories['NO'] == highest_cat_num, '식품명']
    curr_log.loc[i, 'predicted_categories_number'] = highest_cat_num
    curr_log.loc[i, 'predicted_categories_words'] = highest_cat_words.to_list()[0]
    curr_log.loc[i, 'max_cosim_score'] = np.array2string(to_keep[0,-5:])
    curr_log.loc[i, 'most_sim_food'] = most_sim_food_row['식품명']
    curr_log.loc[i, 'reciprocal_rank'] = rr
    curr_log.loc[i, 'sym_reciprocal_rank'] = sym_rr
  return curr_log

In [21]:
# Main method
def food_recommandation(input_food_list) :

  input_list= ["wweia_food_category_code", "Food Name", "wweia_food_category_description"]
  curr_log = pd.DataFrame(input_food_list, columns=input_list)
  
  curr_log = process_food_log(curr_log, wweia_synonym_cats)

  print("true 출력 ")
  print(set(curr_log.loc[:,'wweia_food_category_code'].tolist()))

  print("pred 출력 ")
  print(set(curr_log.loc[:,'predicted_categories_number'].tolist()))
  
  

In [22]:
two_dimensional_array = [
    [1, "메밀전병", "곡류 및 서류"],
    [2, "약식", "곡류 및 서류"],
    [3, "무지개떡", "곡류 및 서류"],
    [45, "초코 마카롱", "과자류"],
    [106, "레몬머랭", "과자류"],
    [105, "코코넛머랭", "과자류"],
    [335, "홍합미역국", "국 및 탕류"]
]
food_recommandation(two_dimensional_array)

-0.08781616389751434
-0.015493202954530716
-0.017064541578292847


C:\Users\gkstk\AppData\Local\Temp\ipykernel_23852\684523104.py:67: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.99717492 0.9972011  0.99731398 0.99734617 0.99830991]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  curr_log.loc[i, 'max_cosim_score'] = np.array2string(to_keep[0,-5:])


-0.21947618573904037
-0.10551625350490212
-0.0896839709021151
-0.09362020716071129
true 출력 
{1, 2, 3, 105, 106, 45, 335}
pred 출력 
{2016, 10, 238, 817, 2289, 56, 702}
